# In this notebook, we mask a word in a text from wikitext, and then use Masked Language Model to unmask the masked word

## Loading the text

In [1]:
from datasets import load_dataset

dataset = load_dataset("wikitext",'wikitext-2-raw-v1')

Found cached dataset wikitext (C:/Users/Abhijeet Gupta/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset["train"][10]["text"]

' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a charac

## Tokenizer 

In [3]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")

text = dataset["train"][10]["text"]

inputs = tokenizer(text, return_tensors="pt")

## Adding Mask to Sentence - Replaced index-5 Tokenized Tensor with tokenized value of '[MASK]'

In [4]:
inputs['input_ids'][0][5]

tensor(2321)

In [5]:
tokenizer.convert_ids_to_tokens(2321, skip_special_tokens=False)

'battle'

### The word that we are masking is 'battle'. Thus, the model must predict a word similar to 'battle'

In [6]:
tokenizer('[MASK]', return_tensors="pt")['input_ids'][0][1]

tensor(103)

In [7]:
inputs['input_ids'][0][5] = tokenizer('[MASK]', return_tensors="pt")['input_ids'][0][1]

## Masked Language Modelling

In [8]:
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-cased")

with torch.no_grad(): 
    logits = model(**inputs).logits
    
# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]


predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'combat'

### The model predicted 'combat' when the masked word was 'battle'. 

# This seems accurate, as 'battle' and 'combat' are synonyms

This is an example of Transfer Learning, in which we use pre-trained models such as 

# Reference :

https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c